# BTC trading algorithm using ANN

Target variable will be rate of change

<p>X1 = (x(t) − x(t − 1))/x(t − 1)</p>
<p>X2 = (x(t) − xl(t))/xh(t) − xl(t)</p>
<p>X3 = (MA(t) − MA(t − 1))/MA(t − 1)</p>
<p>X4 = (MAS(t) − MAL(t))/MAL(t)</p>
<p>X5 = (x(t) − MA(t))/MA(t)</p>
<p>X6 = x(t) − min(x(t), x(t − 1), . . . , x(t − 5))</p>
<p>X7 = x(t) − max(x(t), x(t − 1), . . . , x(t − 5))</p>
<p>X8 = # of days since the last golden-cross</p>
<p>X9 = # of days since the last dead-cross</p>
<p>X10 = the profit while the stock has risen or fallen continuously</p>
<p>X11 = # of days for which the stock has risen or fallen continuously</p>
<p>X12 = MACD(t)</p>
<p>X13 = MACD(ema(t))</p>
<p>X14 = RSI(t)</p>
<p>X15 = %K (t)</p>
<p>X16 = %D(t)</p>
<p>X17 = %K (t) − %K (t − 1 )</p>
<p>X18 = %D(t) − %D(t − 1 )</p>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats

## Load data

In [2]:
#file_name = "Binance_BTCUSDT_1h.csv" # 1 hour data
#file_name = "COINBASE_BTCUSD, 240.csv" # 4 hour data
file_name = "Bitfinex_BTCUSD_d\.csv" # 1 day data
data = pd.read_csv("data/1. Original data/"+file_name)
data.head()

,unix,date,symbol,open,high,low,close,Volume USD,Volume BTC
0,1641427200000,2022-01-06 00:00:00,BTC/USD,43457.000000,43579.000000,43290.0,43556.0,3.132490e+06,71.918690
1,1641340800000,2022-01-05 00:00:00,BTC/USD,45830.000000,47058.000000,42505.0,43455.0,3.012357e+08,6932.129532
2,1641254400000,2022-01-04 00:00:00,BTC/USD,46449.615556,47671.862805,45568.0,45834.0,1.736399e+08,3788.452350
3,1641168000000,2022-01-03 00:00:00,BTC/USD,47304.235105,47578.000000,45689.0,46448.0,1.197406e+08,2577.950091
4,1641081600000,2022-01-02 00:00:00,BTC/USD,47746.000000,48012.000000,46648.0,47302.0,8.725996e+07,1844.741551


## Missing values

In [3]:
data.isnull().sum()

unix          0
date          0
symbol        0
open          0
high          0
low           0
close         0
Volume USD    0
Volume BTC    0
dtype: int64

We wont use variable tradecount, so we can drop it

In [5]:
# If we are using 1H and 1day then we use this format
data = data[['unix', 'date', 'symbol', 'open', 'high', 'low', 'close', 'Volume BTC']]


# If we are using 4H data then we use this format
#data = data[['time', 'open', 'high', 'low', 'close', 'Volume']]
#data.rename({'time': 'unix', 'Volume': 'Volume BTC'}, axis=1, inplace=True)
#data["symbol"] = "BTC/USDT"

## Sort values

In [6]:
# sort the values by symbol and then date
data.sort_values(by = ['unix'], inplace = True)
data = data.reset_index()
data

/var/folders/q4/7jv79_fs2qb85n7v34tqyj040000gn/T/ipykernel_2972/3227026997.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.sort_values(by = ['unix'], inplace = True)


,index,unix,date,symbol,open,high,low,close,Volume BTC
0,2523,1423440000,2015-02-09 00:00:00,BTC/USD,224.220000,225.000000,215.40,220.61,29625.030000
1,2522,1423526400,2015-02-10 00:00:00,BTC/USD,220.610000,223.880000,214.00,220.96,29268.950000
2,2521,1423612800,2015-02-11 00:00:00,BTC/USD,220.960000,224.400000,218.10,219.19,19289.280000
3,2520,1423699200,2015-02-12 00:00:00,BTC/USD,219.190000,223.200000,217.87,222.60,11190.650000
4,2519,1423785600,2015-02-13 00:00:00,BTC/USD,222.390000,241.760000,221.46,236.00,29079.110000
...,...,...,...,...,...,...,...,...,...
2519,4,1641081600000,2022-01-02 00:00:00,BTC/USD,47746.000000,48012.000000,46648.00,47302.00,1844.741551
2520,3,1641168000000,2022-01-03 00:00:00,BTC/USD,47304.235105,47578.000000,45689.00,46448.00,2577.950091
2521,2,1641254400000,2022-01-04 00:00:00,BTC/USD,46449.615556,47671.862805,45568.00,45834.00,3788.452350
2522,1,1641340800000,2022-01-05 00:00:00,BTC/USD,45830.000000,47058.000000,42505.00,43455.00,6932.129532


## X1

In [7]:
# X1 = (x(t) − x(t − 1))/x(t − 1)
X1 = []
for i in range(1, len(data)):
    temp = (data["close"].iloc[i] - data["close"].iloc[i-1]) / data["close"].iloc[i-1]
    X1.append(temp)

data = data[1:]
data["X1"] = X1

/var/folders/q4/7jv79_fs2qb85n7v34tqyj040000gn/T/ipykernel_2972/129941039.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["X1"] = X1


## X2

In [8]:
# X2 = (x(t) − xl(t))/xh(t) − xl(t)
data["X2"] = (data["close"] - data["low"]) / (data["high"] - data["low"])

/var/folders/q4/7jv79_fs2qb85n7v34tqyj040000gn/T/ipykernel_2972/2824311299.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["X2"] = (data["close"] - data["low"]) / (data["high"] - data["low"])


## Sma 50

In [9]:
data["SMA_50"] = data["close"].rolling(window=50).mean()
data["SMA_50_vol"] = data["Volume BTC"].rolling(window=50).mean()

/var/folders/q4/7jv79_fs2qb85n7v34tqyj040000gn/T/ipykernel_2972/2937749957.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["SMA_50"] = data["close"].rolling(window=50).mean()
/var/folders/q4/7jv79_fs2qb85n7v34tqyj040000gn/T/ipykernel_2972/2937749957.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["SMA_50_vol"] = data["Volume BTC"].rolling(window=50).mean()


## Sma 200

In [10]:
data["SMA_200"] = data["close"].rolling(window=200).mean()
data["SMA_200_vol"] = data["Volume BTC"].rolling(window=200).mean()

/var/folders/q4/7jv79_fs2qb85n7v34tqyj040000gn/T/ipykernel_2972/1946598897.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["SMA_200"] = data["close"].rolling(window=200).mean()
/var/folders/q4/7jv79_fs2qb85n7v34tqyj040000gn/T/ipykernel_2972/1946598897.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["SMA_200_vol"] = data["Volume BTC"].rolling(window=200).mean()


## X3

In [11]:
# X3 = (MA(t) − MA(t − 1))/MA(t − 1)
# Short term
X3_1 = []
for i in range(1, len(data)):
    temp = (data["SMA_50"].iloc[i] - data["SMA_50"].iloc[i-1]) / data["SMA_50"].iloc[i-1]
    X3_1.append(temp)

# Long term
X3_2 = []
for i in range(1, len(data)):
    temp = (data["SMA_200"].iloc[i] - data["SMA_200"].iloc[i-1]) / data["SMA_200"].iloc[i-1]
    X3_2.append(temp)
    
# Short term Volume
X3_1_vol = []
for i in range(1, len(data)):
    temp = (data["SMA_50_vol"].iloc[i] - data["SMA_50_vol"].iloc[i-1]) / data["SMA_50_vol"].iloc[i-1]
    X3_1_vol.append(temp)

# Long term Volume
X3_2_vol = []
for i in range(1, len(data)):
    temp = (data["SMA_200_vol"].iloc[i] - data["SMA_200_vol"].iloc[i-1]) / data["SMA_200_vol"].iloc[i-1]
    X3_2_vol.append(temp)

data = data[1:]
data["X3_1"] = X3_1
data["X3_2"] = X3_2
data["X3_1_vol"] = X3_1_vol
data["X3_2_vol"] = X3_2_vol

## X4

In [12]:
# X4 = (MAS(t) − MAL(t))/MAL(t)
data["X4"] = (data["SMA_50"] - data["SMA_200"]) / data["SMA_200"]
data["X4_vol"] = (data["SMA_50_vol"] - data["SMA_200_vol"]) / data["SMA_200_vol"]

## X5

In [13]:
#X5 = (x(t) − MA(t))/MA(t)
data["X5_1"] = (data["close"] - data["SMA_50"]) / data["SMA_50"]
data["X5_2"] = (data["close"] - data["SMA_200"]) / data["SMA_200"]
data["X5_1_vol"] = (data["Volume BTC"] - data["SMA_50_vol"]) / data["SMA_50_vol"]
data["X5_2_vol"] = (data["Volume BTC"] - data["SMA_200_vol"]) / data["SMA_200_vol"]

## X6

In [14]:
# X6 = x(t) − min(x(t), x(t − 1), . . . , x(t − 5))
X6 = []
for i in range(5, len(data)):
    temp = data["close"].iloc[i] - min(data["close"].iloc[i], data["close"].iloc[i-1], data["close"].iloc[i-2], 
               data["close"].iloc[i-3], data["close"].iloc[i-4], data["close"].iloc[i-5])
    X6.append(temp)
    
X6_vol = []
for i in range(5, len(data)):
    temp = data["Volume BTC"].iloc[i] - min(data["Volume BTC"].iloc[i], data["Volume BTC"].iloc[i-1], data["Volume BTC"].iloc[i-2], 
               data["Volume BTC"].iloc[i-3], data["Volume BTC"].iloc[i-4], data["Volume BTC"].iloc[i-5])
    X6_vol.append(temp)
    
data = data[5:]
data["X6"] = X6
data["X6_vol"] = X6_vol

## X7

In [15]:
# X7 = x(t) − max(x(t), x(t − 1), . . . , x(t − 5))
X7 = []
for i in range(5, len(data)):
    temp = data["close"].iloc[i] - max(data["close"].iloc[i], data["close"].iloc[i-1], data["close"].iloc[i-2], 
               data["close"].iloc[i-3], data["close"].iloc[i-4], data["close"].iloc[i-5])
    X7.append(temp)

X7_vol = []
for i in range(5, len(data)):
    temp = data["Volume BTC"].iloc[i] - max(data["Volume BTC"].iloc[i], data["Volume BTC"].iloc[i-1], data["Volume BTC"].iloc[i-2], 
               data["Volume BTC"].iloc[i-3], data["Volume BTC"].iloc[i-4], data["Volume BTC"].iloc[i-5])
    X7_vol.append(temp)
    
data = data[5:]
data["X7"] = X7
data["X7_vol"] = X7_vol

## X8, X9

In [16]:
data["SMA_50"] > data["SMA_200"]

12      False
13      False
14      False
15      False
16      False
        ...  
2519     True
2520     True
2521     True
2522     True
2523     True
Length: 2512, dtype: bool

In [17]:
sma_50 = 0 # Golden cross
sma_200 = 0 # Dead cross
data["Golden_cross"] = 0
data["Dead_cross"] = 0

for i in range(0, len(data)):
    if data["SMA_50"].iloc[i] > data["SMA_200"].iloc[i]:
        sma_200 = 0
        sma_50 += 1
        data["Golden_cross"].iloc[i] = sma_50
        data["Dead_cross"].iloc[i] = sma_200
    else:
        sma_50 = 0
        sma_200 += 1
        data["Golden_cross"].iloc[i] = sma_50
        data["Dead_cross"].iloc[i] = sma_200

data["X8"] = data["Golden_cross"]
data["X9"] = data["Dead_cross"]

/var/folders/q4/7jv79_fs2qb85n7v34tqyj040000gn/T/ipykernel_2972/345647904.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Golden_cross"].iloc[i] = sma_50
/var/folders/q4/7jv79_fs2qb85n7v34tqyj040000gn/T/ipykernel_2972/345647904.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Dead_cross"].iloc[i] = sma_200
/var/folders/q4/7jv79_fs2qb85n7v34tqyj040000gn/T/ipykernel_2972/345647904.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view

In [18]:
sma_50_vol = 0 # Golden cross
sma_200_vol = 0 # Dead cross
data["Golden_cross_vol"] = 0
data["Dead_cross_vol"] = 0

for i in range(0, len(data)):
    if data["SMA_50_vol"].iloc[i] > data["SMA_200_vol"].iloc[i]:
        sma_200 = 0
        sma_50 += 1
        data["Golden_cross_vol"].iloc[i] = sma_50
        data["Dead_cross_vol"].iloc[i] = sma_200
    else:
        sma_50 = 0
        sma_200 += 1
        data["Golden_cross_vol"].iloc[i] = sma_50
        data["Dead_cross_vol"].iloc[i] = sma_200

data["X8_vol"] = data["Golden_cross_vol"]
data["X9_vol"] = data["Dead_cross_vol"]

/var/folders/q4/7jv79_fs2qb85n7v34tqyj040000gn/T/ipykernel_2972/169074001.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Golden_cross_vol"].iloc[i] = sma_50
/var/folders/q4/7jv79_fs2qb85n7v34tqyj040000gn/T/ipykernel_2972/169074001.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Dead_cross_vol"].iloc[i] = sma_200
/var/folders/q4/7jv79_fs2qb85n7v34tqyj040000gn/T/ipykernel_2972/169074001.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

## X10

In [19]:
(data["close"] / data["open"] ) -1

12     -0.001751
13     -0.034721
14      0.013272
15     -0.000209
16     -0.007009
          ...   
2519   -0.009299
2520   -0.018101
2521   -0.013253
2522   -0.051822
2523    0.002278
Length: 2512, dtype: float64

In [20]:
up_days_profit = 0.0
down_days_profit = 0.0
data["Up_Days_Profit"] = 0.0
data["Down_Days_Profit"] = 0.0

for i in range(0, len(data)):
    if data["close"].iloc[i] > data["open"].iloc[i]: # UP days
        down_days_profit = 0.0
        up_days_profit += ((data["close"].iloc[i] / data["open"].iloc[i] ) -1)
        data["Up_Days_Profit"].iloc[i] = up_days_profit
        data["Down_Days_Profit"].iloc[i] = down_days_profit
    else: # DOWN days
        up_days_profit = 0.0
        down_days_profit += ((data["close"].iloc[i] / data["open"].iloc[i] ) -1)
        data["Up_Days_Profit"].iloc[i] = up_days_profit
        data["Down_Days_Profit"].iloc[i] = down_days_profit

data["X10_1"] = data["Up_Days_Profit"]
data["X10_2"] = data["Down_Days_Profit"]

/var/folders/q4/7jv79_fs2qb85n7v34tqyj040000gn/T/ipykernel_2972/1808392782.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Up_Days_Profit"].iloc[i] = up_days_profit
/var/folders/q4/7jv79_fs2qb85n7v34tqyj040000gn/T/ipykernel_2972/1808392782.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Down_Days_Profit"].iloc[i] = down_days_profit
/var/folders/q4/7jv79_fs2qb85n7v34tqyj040000gn/T/ipykernel_2972/1808392782.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/in

## X11

In [21]:
up_days = 0
down_days = 0
data["Up_Days"] = 0
data["Down_Days"] = 0

for i in range(0, len(data)):
    if data["close"].iloc[i] > data["open"].iloc[i]: # UP days
        down_days = 0
        up_days += 1
        data["Up_Days"].iloc[i] = up_days
        data["Down_Days"].iloc[i] = down_days
    else: # DOWN days
        up_days = 0
        down_days += 1
        data["Up_Days"].iloc[i] = up_days
        data["Down_Days"].iloc[i] = down_days

data["X11_1"] = data["Up_Days"]
data["X11_2"] = data["Down_Days"]

/var/folders/q4/7jv79_fs2qb85n7v34tqyj040000gn/T/ipykernel_2972/975969267.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Up_Days"].iloc[i] = up_days
/var/folders/q4/7jv79_fs2qb85n7v34tqyj040000gn/T/ipykernel_2972/975969267.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Down_Days"].iloc[i] = down_days
/var/folders/q4/7jv79_fs2qb85n7v34tqyj040000gn/T/ipykernel_2972/975969267.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ve

## X12, X13

MACD

In [22]:
# Calculate the MACD
ema_26 = data['close'].transform(lambda x: x.ewm(span = 26).mean())
ema_12 = data['close'].transform(lambda x: x.ewm(span = 12).mean())
macd = ema_12 - ema_26

# Calculate the EMA
ema_9_macd = macd.ewm(span = 9).mean()

# Store the data in the data frame.
data['MACD'] = macd
data['MACD_EMA'] = ema_9_macd

# Print the head.
data["X12"] = data["MACD"]
data["X13"] = data["MACD_EMA"]

In [23]:
# Calculate the MACD
ema_26 = data['Volume BTC'].transform(lambda x: x.ewm(span = 26).mean())
ema_12 = data['Volume BTC'].transform(lambda x: x.ewm(span = 12).mean())
macd = ema_12 - ema_26

# Calculate the EMA
ema_9_macd = macd.ewm(span = 9).mean()

# Store the data in the data frame.
data['MACD_vol'] = macd
data['MACD_EMA_vol'] = ema_9_macd

# Print the head.
data["X12_vol"] = data["MACD_vol"]
data["X13_vol"] = data["MACD_EMA_vol"]

## X14

RSI

In [24]:
# calculate the change in price
data['change_in_price'] = data['close'].diff()
data.head()
# Calculate the 14 day RSI
n = 14

# First make a copy of the data frame twice
up_df, down_df = data[['symbol','change_in_price']].copy(), data[['symbol','change_in_price']].copy()

# For up days, if the change is less than 0 set to 0.
up_df.loc['change_in_price'] = up_df.loc[(up_df['change_in_price'] < 0), 'change_in_price'] = 0

# For down days, if the change is greater than 0 set to 0.
down_df.loc['change_in_price'] = down_df.loc[(down_df['change_in_price'] > 0), 'change_in_price'] = 0

# We need change in price to be absolute.
down_df['change_in_price'] = down_df['change_in_price'].abs()

# Calculate the EWMA (Exponential Weighted Moving Average), meaning older values are given less weight compared to newer values.
ewma_up = up_df.groupby('symbol')['change_in_price'].transform(lambda x: x.ewm(span = n).mean())
ewma_down = down_df.groupby('symbol')['change_in_price'].transform(lambda x: x.ewm(span = n).mean())

# Calculate the Relative Strength
relative_strength = ewma_up / ewma_down

# Calculate the Relative Strength Index
relative_strength_index = 100.0 - (100.0 / (1.0 + relative_strength))

# Add the info to the data frame.
data['down_days'] = down_df['change_in_price']
data['up_days'] = up_df['change_in_price']
data['RSI'] = relative_strength_index

data["X14"] = data["RSI"]


## X15

Stochastic Oscillator

In [25]:
# Calculate the Stochastic Oscillator
n = 14

# Make a copy of the high and low column.
low_14, high_14 = data[['symbol','low']].copy(), data[['symbol','high']].copy()

# Group by symbol, then apply the rolling function and grab the Min and Max.
low_14 = low_14.groupby('symbol')['low'].transform(lambda x: x.rolling(window = n).min())
high_14 = high_14.groupby('symbol')['high'].transform(lambda x: x.rolling(window = n).max())

# Calculate the Stochastic Oscillator.
k_percent = 100 * ((data['close'] - low_14) / (high_14 - low_14))

# Add the info to the data frame.
data['low_14'] = low_14
data['high_14'] = high_14
data['k_percent'] = k_percent

data["X15"] = data["k_percent"]

## X16

D% = K$ ma 14

In [26]:
data["d_percent"] = data["k_percent"].rolling(window=14).mean()

In [27]:
data["X16"] = data["d_percent"]

## X17

In [28]:
# X17 = %K (t) − %K (t − 1 )
#data["X16"] = 0
X17 = []
for i in range(1, len(data)):
    temp = data["k_percent"].iloc[i] - data["k_percent"].iloc[i-1]
    X17.append(temp)
data = data[1:]
data["X17"] = X17

/var/folders/q4/7jv79_fs2qb85n7v34tqyj040000gn/T/ipykernel_2972/2978392693.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["X17"] = X17


## X18

In [29]:
# X18 = %D(t) − %D(t − 1 )
X18 = []
for i in range(1, len(data)):
    temp = data["d_percent"].iloc[i] - data["d_percent"].iloc[i-1]
    X18.append(temp)
data = data[1:]
data["X18"] = X18

/var/folders/q4/7jv79_fs2qb85n7v34tqyj040000gn/T/ipykernel_2972/2572392827.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["X18"] = X18


## Target variable

In [30]:
# [x(t+1)−x(t)]  /  x(t)
target = []
for i in range(0, len(data)-1):
    temp = (data["close"].iloc[i+1] - data["close"].iloc[i]) / data["close"].iloc[i]
    target.append(temp)
data = data[:-1]
data["target"] = target

In [31]:
data

,index,unix,date,symbol,open,high,low,close,Volume BTC,X1,...,X14,low_14,high_14,k_percent,X15,d_percent,X16,X17,X18,target
14,2509,1424649600,2015-02-23 00:00:00,BTC/USD,236.590000,241.000000,232.61,239.73,23893.090000,0.013272,...,29.861155,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.000209
15,2508,1424736000,2015-02-24 00:00:00,BTC/USD,239.730000,240.990000,236.70,239.68,16784.310000,-0.000209,...,29.698216,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.007009
16,2507,1424822400,2015-02-25 00:00:00,BTC/USD,239.680000,240.830000,235.53,238.00,13574.500000,-0.007009,...,24.512651,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.004202
17,2506,1424908800,2015-02-26 00:00:00,BTC/USD,238.000000,238.340000,233.62,237.00,18691.430000,-0.004202,...,21.887790,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.081857
18,2505,1424995200,2015-02-27 00:00:00,BTC/USD,237.000000,262.800000,236.64,256.40,69269.580000,0.081857,...,77.005441,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.002457
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2518,5,1640995200000,2022-01-01 00:00:00,BTC/USD,46209.000000,47957.000000,46203.00,47747.00,2177.337470,0.033082,...,45.918504,45600.000000,52085.0,33.107170,33.107170,48.936931,48.936931,22.515887,0.869086,-0.009320
2519,4,1641081600000,2022-01-02 00:00:00,BTC/USD,47746.000000,48012.000000,46648.00,47302.00,1844.741551,-0.009320,...,43.216147,45600.000000,52085.0,26.245181,26.245181,49.503115,49.503115,-6.861989,0.566185,-0.018054
2520,3,1641168000000,2022-01-03 00:00:00,BTC/USD,47304.235105,47578.000000,45689.00,46448.00,2577.950091,-0.018054,...,38.233655,45664.143622,52085.0,12.207972,12.207972,48.848379,48.848379,-14.037209,-0.654736,-0.013219
2521,2,1641254400000,2022-01-04 00:00:00,BTC/USD,46449.615556,47671.862805,45568.00,45834.00,3788.452350,-0.013219,...,34.896048,45568.000000,52085.0,4.081633,4.081633,44.917790,44.917790,-8.126340,-3.930589,-0.051905


## Combining data

In [32]:
data.columns

Index(['index', 'unix', 'date', 'symbol', 'open', 'high', 'low', 'close',
       'Volume BTC', 'X1', 'X2', 'SMA_50', 'SMA_50_vol', 'SMA_200',
       'SMA_200_vol', 'X3_1', 'X3_2', 'X3_1_vol', 'X3_2_vol', 'X4', 'X4_vol',
       'X5_1', 'X5_2', 'X5_1_vol', 'X5_2_vol', 'X6', 'X6_vol', 'X7', 'X7_vol',
       'Golden_cross', 'Dead_cross', 'X8', 'X9', 'Golden_cross_vol',
       'Dead_cross_vol', 'X8_vol', 'X9_vol', 'Up_Days_Profit',
       'Down_Days_Profit', 'X10_1', 'X10_2', 'Up_Days', 'Down_Days', 'X11_1',
       'X11_2', 'MACD', 'MACD_EMA', 'X12', 'X13', 'MACD_vol', 'MACD_EMA_vol',
       'X12_vol', 'X13_vol', 'change_in_price', 'down_days', 'up_days', 'RSI',
       'X14', 'low_14', 'high_14', 'k_percent', 'X15', 'd_percent', 'X16',
       'X17', 'X18', 'target'],
      dtype='object')

In [33]:
data_ann = data[['unix', 'open', 'high', 'low', 'close', 'X1', 'X2', 'X3_1', 'X3_2', 'X3_1_vol', 
                 'X3_2_vol', 'X4','X4_vol', 'X5_1', 'X5_2', 'X5_1_vol', 'X5_2_vol', 'X6', 'X6_vol', 
                 'X7','X7_vol', 'X8', 'X9','X8_vol','X9_vol','X10_1','X10_2','X11_1','X11_2', 'X12', 
                 'X13', 'X12_vol', 'X13_vol','X14', 'X15', 'X16','X17', 'X18', 'target']]
data_ann.tail()

,unix,open,high,low,close,X1,X2,X3_1,X3_2,X3_1_vol,...,X12,X13,X12_vol,X13_vol,X14,X15,X16,X17,X18,target
2518,1640995200000,46209.000000,47957.000000,46203.0,47747.0,0.033082,0.880274,-0.006180,0.000795,-0.010271,...,-1254.094786,-1355.940704,-741.802792,-694.998623,45.918504,33.107170,48.936931,22.515887,0.869086,-0.009320
2519,1641081600000,47746.000000,48012.000000,46648.0,47302.0,-0.009320,0.479472,-0.006488,0.000935,-0.000083,...,-1231.588154,-1331.070194,-840.365830,-724.072064,43.216147,26.245181,49.503115,-6.861989,0.566185,-0.018054
2520,1641168000000,47304.235105,47578.000000,45689.0,46448.0,-0.018054,0.401800,-0.007267,0.000874,0.001352,...,-1268.044916,-1318.465138,-849.521194,-749.161890,38.233655,12.207972,48.848379,-14.037209,-0.654736,-0.013219
2521,1641254400000,46449.615556,47671.862805,45568.0,45834.0,-0.013219,0.126434,-0.006821,0.001041,-0.005225,...,-1331.137309,-1320.999572,-750.448764,-749.419265,34.896048,4.081633,44.917790,-8.126340,-3.930589,-0.051905
2522,1641340800000,45830.000000,47058.000000,42505.0,43455.0,-0.051905,0.208654,-0.006436,0.000821,-0.021729,...,-1555.176793,-1367.835016,-413.498125,-682.235037,25.100224,9.916493,41.460675,5.834860,-3.457115,0.002324


## Save in .csv

In [34]:
data_ann.to_csv("data/2. Prepared data/"+file_name, index=False)